## Introduction

### What is Function Calling

Function calling enables Large Language Models (LLMs) to interact with external systems, execute tasks, and integrate with APIs. The LLM determines when to invoke a function based on user prompts and returns structured data for app use. Developers then implement the function logic within the app.

In this workshop, the function logic is used to execute the LLM dynamically generated SQL queries against the SQLite database.

### Enabling Function Calling

If you’re familiar with Azure OpenAI Function Calling, it requires defining a function schema for the LLM. Azure AI Agent Service supports this approach and also offers a more flexible option.

With the Azure AI Agent Service and its Python SDK, you can define the function schema directly within the Python function’s docstring. This approach keeps the definition and implementation together, simplifying maintenance and enhancing readability.

For example, in the sales_data.py file, the async_fetch_sales_data_using_sqlite_query function uses a docstring to specify its signature, inputs, and outputs. The SDK parses this docstring to generate the callable function for the LLM:

```python
async def async_fetch_sales_data_using_sqlite_query(self: "SalesData", sqlite_query: str) -> str:
    """
    This function is used to answer user questions about Contoso sales data by executing SQLite queries against the database.

    :param sqlite_query: The input should be a well-formed SQLite query to extract information based on the user's question. The query result will be returned as a JSON object.
    :return: Return data in JSON serializable format.
    :rtype: str
    """
```

In [ ]:
import os
import sys
from dotenv import load_dotenv


# get current folder
current_directory = os.getcwd()
# get parent folder
parent_directory = os.path.abspath(os.path.join(current_directory, ".."))
sys.path.append(parent_directory)

project_directory = os.path.abspath(os.path.join(parent_directory, ".."))
sys.path.append(project_directory)

load_dotenv('.env')

In [ ]:
from main import initialize
from azure.ai.projects.aio import AIProjectClient
from azure.ai.projects.models import (
    Agent,
    AgentThread,
    AsyncFunctionTool,
    AsyncToolSet,
    BingGroundingTool,
    CodeInterpreterTool,
    FileSearchTool,
)
from azure.identity import DefaultAzureCredential
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry import trace
from sales_data import SalesData
from stream_event_handler import StreamEventHandler
from terminal_colors import TerminalColors as tc
from utilities import Utilities


In [ ]:
TENTS_DATA_SHEET_FILE = "/home/azureuser/azure-ai-agent-workshop/azure-ai-agent-service-sampleCode/datasheet/contoso-tents-datasheet.pdf"
API_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT")
PROJECT_CONNECTION_STRING = os.environ["PROJECT_CONNECTION_STRING"]
BING_CONNECTION_NAME = os.getenv("BING_CONNECTION_NAME")
MAX_COMPLETION_TOKENS = 4096
MAX_PROMPT_TOKENS = 10240
TEMPERATURE = 0.2

toolset = AsyncToolSet()
sales_data = SalesData()
utilities = Utilities()

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=PROJECT_CONNECTION_STRING,
)

functions = AsyncFunctionTool(
    {
        sales_data.async_fetch_sales_data_using_sqlite_query,
    }
)


### Dynamic SQL Generation

When the app starts, it incorporates the database schema and key data into the instructions for the Azure AI Agent Service. Using this input, the LLM generates SQLite-compatible SQL queries to respond to user requests expressed in natural language.

### Lab Exercise

In this lab, you'll enable the function logic to execute dynamic SQL queries against the SQLite database. The function will be called by the LLM to answer user questions about Contoso sales data.

### Run the Agent App

Start a Conversation with the Agent

Here is an example of the LLM response to the help query:

```
I’m here to help with your sales data inquiries at Contoso. Could you please provide more details about what you need assistance with? Here are some example queries you might consider:

- "What were the sales by region?"
- "What was last quarter's revenue?"
- "Which products sell best in Europe?"
- "Total shipping costs by region?"

Feel free to ask any specific questions related to Contoso sales data!
```

2. What are the sales by region?

Here is an example of the LLM response to the sales by region query:

```markdown
| Region         | Total Revenue  |
|----------------|----------------|
| AFRICA         | $5,227,467     |
| ASIA-PACIFIC   | $5,363,718     |
| CHINA          | $10,540,412    |
| EUROPE         | $9,990,708     |
| LATIN AMERICA  | $5,386,552     |
| MIDDLE EAST    | $5,312,519     |
| NORTH AMERICA  | $15,986,462    |
```

In [12]:
from terminal_colors import TerminalColors as tc
from main import post_message, cleanup

INSTRUCTIONS_FILE = "/home/azureuser/azure-ai-agent-workshop/azure-ai-agent-service-sampleCode/instructions/instructions_function_calling.txt"

agent, thread = await initialize()

while True:
    # Get user input prompt in the terminal using a pretty shade of green
    print("\n")
    prompt = input(f"{tc.GREEN}Enter your query (type exit to finish): {tc.RESET}")
    if prompt.lower() == "exit":
        break
    if not prompt:
        continue
    await post_message(agent=agent, thread_id=thread.id, content=prompt, thread=thread)

await cleanup(agent, thread)

Uploading file: /home/azureuser/azure-ai-agent-workshop/azure-ai-agent-service-sampleCode/datasheet/contoso-tents-datasheet.pdf
Creating the vector store
Vector store created and files added.
Creating agent...
Created agent, ID: asst_Q2s2w2s6yAIqBFuMe3ZXARpv
Creating thread...
Created thread, ID: thread_u73kYcMjPid387AblAuMW1sC



Function Call Tools: async_fetch_sales_data_using_sqlite_query

Executing query: SELECT region, SUM(revenue) AS total_revenue, SUM(shipping_cost) AS total_shipping_cost, SUM(number_of_orders) AS total_orders FROM sales_data GROUP BY region LIMIT 30;

Here are the sales by region:

| Region          | Total Revenue | Total Shipping Cost | Total Orders |
|------------------|---------------|---------------------|--------------|
| AFRICA           | $5,227,467    | $773,515.89         | 37,675       |
| ASIA-PACIFIC     | $5,363,718    | $806,608.01         | 37,318       |
| CHINA            | $10,540,412   | $1,589,962.50       | 74,382       |
| EUROPE        